In [3]:
import numpy as np
import pandas as pd
import pickle
import re
import nltk
import nltk.corpus
import numpy as np
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.snowball import SnowballStemmer
plt.rcParams["figure.figsize"] = [10, 8]

In [3]:
def tokenize_text(doc):
    """Combine the strings in the "response" column of dataframe df into one long string. Then, tokenize the
    string and make all words lowercase."""

    # Tokenize and make lowercase.
    words = nltk.word_tokenize(doc)
    words = [w.lower() for w in words]
    
    return words


def wordnet_pos(tag):
    """Map a Brown POS tag to a WordNet POS tag."""
    
    table = {"N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV, "J": wordnet.ADJ}
    
    # Default to a noun.
    return table.get(tag[0], wordnet.NOUN)


def lemmatize_text(words):
    """Lemmatize words to get the base words. The input 'words' is a list of of words."""
    
    lemmatizer = nltk.WordNetLemmatizer()
    word_tags = nltk.pos_tag(words)
    words = [lemmatizer.lemmatize(w, wordnet_pos(t)) for (w, t) in word_tags]
    
    return words


def remove_stopwords(words):
    """Remove stopwords from a string."""
    
    stopwords = nltk.corpus.stopwords.words("english")
    words = [w for w in words if w not in stopwords]
    
    return words

def clean_text(doc):    
    """Tokenize, lemmatize, and remove stopwords for the text of all articles."""
    
    words = re.sub("< ?/?[a-z]+ ?>|\n", "", doc)
    words = tokenize_text(words)
    words = lemmatize_text(words)
    words = remove_stopwords(words)
    doc = [w for w in words if w.isalnum()]
    doc = ' '.join(doc)
    
    return doc

def clean_df(df):
    """Combine the title and content of each post into one string and clean each string."""
    text = df['title'] + " " + df['content']
    df_clean = pd.DataFrame([clean_text(i) for i in text])
    df_clean.columns = ["text"]
    #df_clean["tags"] = df["tags"]
    df_clean = pd.concat([df_clean, pd.DataFrame(df["tags"])],axis = 1, sort = False)
    return df_clean

In [3]:
## Functions from examine_tags.ipynb
def get_top_tags(df):
    tag_list = [tags.split(' ') for tags in df['tags']]
    flat_list = [item for sublist in tag_list for item in sublist]
    fq = nltk.FreqDist(w for w in flat_list)
    df_fq = pd.DataFrame.from_dict(fq, orient="index").reset_index()
    df_95 = df_fq[df_fq.iloc[:,1] >= np.percentile(df_fq.iloc[:,1],95)].reset_index(drop=True)
    df_95.columns = ["term", "fq"]
    return df_95

def subset_top_df(df, top_tags_df):
    """
    df: DataFrame with all posts
    top_tags_df: Data frame of top tags
    """
    tags_list = [tags.split(' ') for tags in df['tags']]
    indeces = [i for i in range(len(tags_list))
               if list(set(top_tags_df['term']) & 
                       set(tags_list[i])) != []]
    return df.loc[indeces]

In [4]:
def trim_tags_clean(df):
    top = get_top_tags(df)
    top_subset = subset_top_df(df, top).reset_index()
    return clean_df(top_subset)

In [ ]:
# Stack exchange topic names
names = ["biology","cooking","crypto","diy","robotics","travel"]

def get_paths(name):
    """Get path names for each file."""
    path = "data/"+name+".csv"
    return path

# Get path names
paths = [get_paths(i) for i in names]

# All data frames in a list.
dfs = [pd.read_csv(i) for i in paths]

# Get a list of the cleaned data frames.
trim_clean_dfs = [trim_tags_clean(i) for i in dfs]

# Save cleaned dfs as csv
for i in range(len(names)):
    trim_clean_dfs[i].to_pickle(names[i]+"_trim_clean.dat")

In [14]:
data = pd.read_pickle("~/Downloads/robotics_finalDF")
data_all = pd.read_pickle("~/Downloads/full_finalDF")

In [15]:
data_all

,tags,text,120-240v,240v,2nd-preimage-resistance,3d-reconstruction,3d-structure,3des,90-180-visa-rules,abe,...,zero-knowledge-proofs,zika,zimbabwe,zimbabwean-citizen,zoology,zoos,zqn,zrh,zucchini,zurich
0,loyalty-programs routes ewr singapore-airlines...,singapore airline offer reward seat route sing...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,romania transportation,easy transportation use throughout romania for...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,usa airport-transfer taxis seattle,best way get seatac airport redmond anyone sug...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,sightseeing public-transport transportation ar...,destination first time trip argentina consider...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,safety international-travel money exchange,travel country different currency take money r...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,russia visas china mongolia trans-siberian,best way obtain visa railway plan take moscow ...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,online-resources transportation peru south-ame...,find information roadblock strike peru bolivia...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,us-citizens travel-agents cuba,advisable u citizen attempt visit cuba go agen...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,sightseeing hungary budapest,sight see budapest hungary plan first internat...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,europe online-resources planning guides trains,good website plan trip via train europe wife d...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
# test
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, random_state=42, test_size=0.30, shuffle=True)
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,3), norm='l2')

train_text = train['text']
test_text = test['text']

vectorizer.fit(train_text)
vectorizer.fit(test_text)

x_train = vectorizer.transform(train_text)
y_train = train.drop(labels = ['text'], axis=1)

x_test = vectorizer.transform(test_text)
y_test = test.drop(labels = ['text'], axis=1)

In [8]:
split_list = [tags.split(" ") for tags in list(data['tags'])]
split = [item for sublist in split_list for item in sublist]
unique = list(set(split))
categories = unique

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier

# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])

for category in categories:
    print('**Processing {} comments...**'.format(category))
    
    # Training logistic regression model on train data
    LogReg_pipeline.fit(x_train, train[category])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(x_test)
    #print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))
    #print("\n")
    #f1_score(test[category], prediction,average='weighted')

**Processing lidar comments...**


NameError: name 'f1_score' is not defined

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
# Using pipeline for applying logistic regression and one vs rest classifier
LogReg_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=-1)),
            ])
for category in categories:
    print('**Processing {} comments...**'.format(category))
    
    # Training logistic regression model on train data
    #LogReg_pipeline.fit(x_train, train[category])
    LogReg_pipeline.fit(x_train, train[category])
    
    # calculating test accuracy
    prediction = LogReg_pipeline.predict(x_test)
    print('Test accuracy is {}'.format(accuracy_score(test[category], prediction)))
    print("\n")

**Processing lidar comments...**
Test accuracy is 0.9961538461538462


**Processing routing comments...**
Test accuracy is 1.0


**Processing battle-bot comments...**
Test accuracy is 0.9980769230769231


**Processing errors comments...**
Test accuracy is 0.9980769230769231


**Processing reference-request comments...**
Test accuracy is 0.9980769230769231


**Processing protection comments...**
Test accuracy is 1.0


**Processing c comments...**
Test accuracy is 0.9961538461538462


**Processing kinematics comments...**
Test accuracy is 0.9807692307692307


**Processing vex comments...**
Test accuracy is 1.0


**Processing auv comments...**
Test accuracy is 1.0


**Processing kinect comments...**
Test accuracy is 0.9865384615384616


**Processing microcontroller comments...**
Test accuracy is 0.9403846153846154


**Processing stereo-vision comments...**
Test accuracy is 0.9923076923076923


**Processing mindstorms comments...**
Test accuracy is 1.0


**Processing dynamics comments...**

KeyboardInterrupt: 

In [16]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier.fit(x_train, y_train)
# predict
predictions = classifier.predict(x_test)
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("\n")

TypeError: no supported conversion for types: (dtype('O'),)